In [1]:
#import dependencies and assign API key
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

eia_key = "3573bd97083abd7efae3399626d0ae59"

In [2]:
#identify series names and series ID for API data series call

#category API call - get series IDs for state level gasoline consumption
category_url = f"http://api.eia.gov/category/?api_key={eia_key}&category_id=404254"

#requests to get data from category url
data=requests.get(category_url).json()

#create empty lists for series_id and series_name
series_id=[]
series_name=[]

#iterate over the category and child series to extract 
for x in range(0, len(data['category']['childseries'])-1):
    series_id.append(data['category']['childseries'][x]['series_id'])
    series_name.append(data['category']['childseries'][x]['name'])

In [3]:
#place series ID and series names into a Dataframe
series_df = pd.DataFrame()
series_df["series_id"] = series_id
series_df["state summary"] = series_name
series_df.head()

,series_id,state summary
0,PET.C100000001.A,U.S. Total Gasoline All Sales/Deliveries by Pr...
1,PET.C100000001.M,U.S. Total Gasoline All Sales/Deliveries by Pr...
2,PET.C100010001.A,East Coast (PADD 1) Total Gasoline All Sales/D...
3,PET.C100010001.M,East Coast (PADD 1) Total Gasoline All Sales/D...
4,PET.C100011001.A,New England (PADD 1A) Total Gasoline All Sales...


In [4]:
#remove annual data from series_id's
remove_annual_df = series_df[~series_df["series_id"].str.contains(".A")]

#remove PADD level data (we want state only)
remove_padd_df = remove_annual_df[~remove_annual_df["state summary"].str.contains("PADD")]

#rename index and create final column that extracts state name from the state summary
remove_padd_df = remove_padd_df.reset_index(drop=True)
remove_padd_df["state"] = remove_padd_df["state summary"].str.split('Total').str[0]
remove_padd_df.tail()

,series_id,state summary,state
46,PET.C100050041.M,Arizona Total Gasoline All Sales/Deliveries by...,Arizona
47,PET.C100050061.M,California Total Gasoline All Sales/Deliveries...,California
48,PET.C100050151.M,Hawaii Total Gasoline All Sales/Deliveries by ...,Hawaii
49,PET.C100050321.M,Nevada Total Gasoline All Sales/Deliveries by ...,Nevada
50,PET.C100050411.M,Oregon Total Gasoline All Sales/Deliveries by ...,Oregon


In [5]:
#identify iterable series IDs from remove_padd_df  
iter_ids = remove_padd_df["series_id"].values

In [6]:
#create empty list of dataframes
new_dfs=[]

for id in iter_ids:
    
    #query API based on series ID
    data_url = f"http://api.eia.gov/series/?api_key={eia_key}&series_id={id}"
    
#     #collect series data from specific url from id
    series_data = requests.get(data_url).json()
    
    st_cons = series_data['series'][0]['data']
    df = pd.DataFrame(np.array(st_cons))
    state_name = series_data['series'][0]['name'].split("Total")
    df = df.rename(columns={0:"Month"})
    df = df.rename(columns={1:state_name[0]})
    df = df.set_index("Month")
    new_dfs.append(df)

In [9]:
#merge entire list of individual state level dataframes
state_cons_df = pd.DataFrame()
state_cons_df = pd.concat(new_dfs, axis=1, sort=True)
state_cons_df = state_cons_df.T
state_cons_df = state_cons_df.loc[:, '201601':]
state_cons_df.head()

Month,201601,201602,201603,201604,201605,201606,201607,201608,201609,201610,...,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812
U.S.,342192.8,360546.8,366846.1,370840.6,373972.4,389669.5,385165.4,384718.7,375497.9,373717.8,...,374065.7,374734.3,386016,391639.9,383738.2,390031.8,371360.6,378336.5,372857.2,363814.8
Connecticut,4176.3,4337.7,4398.2,4606.9,4484.6,4408.3,4455.5,4290.5,4195.5,4223.5,...,4171,4196.5,4449.4,4479.9,4355.7,4359.6,4151.3,4165.6,4227.6,4226.8
Maine,1771.2,1785.3,1763.6,1857,1960.6,2091.3,2243.5,2252.3,2086.1,1986.5,...,1821.6,1796.7,2024.7,2164.7,2193.5,2266.8,2034.3,2075.5,1904.4,1877.8
Massachusetts,6563.6,6678.8,6871.9,7083.4,7011.9,7783.8,7810.1,7659.2,7615.9,7567.7,...,6194.4,6361.5,6694.4,6793.7,6674.5,6797.6,6475.5,6578.2,6459.8,6387
New Hampshire,1531.1,1572.9,1517.3,1489.8,1583.2,1595.2,1660.5,1641.1,1541.7,1541.5,...,1420.1,1419.1,1462.2,1626.5,1625,1656.1,1538.2,1612.4,1535.7,1553.5


In [11]:
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.types import VARCHAR

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float

from sqlalchemy import create_engine

dbuser = 'root'
dbpassword = 'ro-Cha7t'
dbhost = 'localhost'
dbport = '3306'
dbname= 'ETL_db'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")
# Base.metadata.create_all(engine)

# state_cons_df
state_cons_df.to_sql('cons', engine, index=True, if_exists="append")